In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten, LeakyReLU, BatchNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16

from sklearn.metrics import accuracy_score, confusion_matrix

from PIL import Image 
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
LANDSCAPE_TYPE = 'buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'

# configurar de acuerdo a dónde bajaron los sets de imágenes
TRAIN_DIR = Path('./ucse-ia-2024-tp-2-clasificacion-de-imagenes/train')
TEST_DIR = Path('./ucse-ia-2024-tp-2-clasificacion-de-imagenes/test')

SIZE = 100

In [ ]:
images_reader = ImageDataGenerator(
    rescale=1/255,  # Normaliza los píxeles a valores entre 0 y 1
    rotation_range=20,  # Rotar imágenes aleatoriamente entre 0 y 20 grados
    width_shift_range=0.2,  # Desplazar horizontalmente hasta el 20% del ancho
    height_shift_range=0.2,  # Desplazar verticalmente hasta el 20% de la altura
    shear_range=0.2,  # Cortar (shear) imágenes aleatoriamente
    zoom_range=0.2,  # Ampliar imágenes aleatoriamente
    horizontal_flip=True,  # Voltear imágenes horizontalmente
    fill_mode='nearest',  # Estrategia para rellenar nuevos píxeles
    brightness_range=(0.5, 1.5),  # Cambiar brillo aleatoriamente
    validation_split=0.4  # Porcentaje para la validación (40%)
)

READ_PARAMS = dict(
    class_mode="categorical",
    classes=LANDSCAPE_TYPE, # para usar el mismo orden en todos lados
    target_size=(SIZE, SIZE), # para que corra más rápido, vamos a achicar las imágenes
    color_mode="rgb",  # queremos trabajar con las imágenes a color
    batch_size=32, # Tamaño del lote
)

In [ ]:
# Generador para entrenamiento (60%)
train = images_reader.flow_from_directory(
    TRAIN_DIR,
    subset='training',
    **READ_PARAMS
)

# Generador para validación (40%)
validation = images_reader.flow_from_directory(
    TRAIN_DIR,
    subset='validation',
    **READ_PARAMS
)

In [ ]:
def sample_images_train(dataset):
    plt.figure(figsize=(10, 10))
    images, labels = next(dataset)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(LANDSCAPE_TYPE[np.argmax(labels[i])])
        plt.axis("off")

In [ ]:
sample_images_train(train)

# 1 Análisis exploratorio sobre el conjunto de datos

## 1.1 Volumetría de los datos

In [ ]:
def count_images_by_category(dataset_dir):
    categories = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    category_counts = {}

    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        image_files = [f for f in os.listdir(category_path) if f.endswith(('jpg', 'jpeg', 'png'))]
        category_counts[category] = len(image_files)

    total_images = sum(category_counts.values())

    return category_counts, total_images

# Obtener la volumetría de los datos
category_counts, total_images = count_images_by_category(TRAIN_DIR)

print("Volumetría del dataset:")
print(f"Total de imágenes: {total_images}")
for category, count in category_counts.items():
    print(f"{category}: {count} imágenes")

## 1.2 Estructura y tipo de las imágenes

In [ ]:
def get_image_dimensions(dataset_dir):
    image_dimensions = []
    image_formats = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                image_dimensions.append(img.size)  # Obtiene el tamaño (ancho, alto)
                image_formats.append(img.format)  # Obtiene el formato de la imagen
    
    return image_dimensions, image_formats

# Obtener la estructura de las imágenes
image_dimensions, image_formats = get_image_dimensions(TRAIN_DIR)

# Muestra algunos ejemplos de dimensiones y formatos
print("Ejemplos de dimensiones de imágenes:")
for i in range(5):
    print(f"Imagen {i+1}: {image_dimensions[i]} (Formato: {image_formats[i]})")


## 1.3 Distribución de la variable a predecir

In [ ]:
def plot_image_distribution(category_counts):
    categories = list(category_counts.keys())
    counts = list(category_counts.values())

    plt.figure(figsize=(10, 6))
    plt.bar(categories, counts, color='skyblue')
    plt.xlabel('Categorías')
    plt.ylabel('Número de imágenes')
    plt.title('Distribución de imágenes por categoría')
    plt.xticks(rotation=45)
    plt.show()

# Graficar la distribución de las imágenes por categoría
plot_image_distribution(category_counts)

# 2 Modelado

In [ ]:
# UTILS

model_weights_at_epochs = {}

class OurCustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_weights_at_epochs[epoch] = self.model.get_weights()

# Función para plotear el historial de entrenamiento
def PlotHistory(history):
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.title('Accuracy over train epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()

class MLPBuilder:
    def __init__(
        self, 
        input_shape=(SIZE, SIZE, 3), 
        output_size=len(LANDSCAPE_TYPE),
    ):
        self.input_shape = input_shape  # Ej: (SIZE, SIZE, 3)
        self.output_size = output_size  # Ej: len(LANDSCAPE_TYPE)
        self.model = None
    
    def build_model(self, layers_config, batch_normalization=False):
        """
        Construye un modelo secuencial MLP basado en la configuración proporcionada.

        :param layers_config: Lista de diccionarios con la configuración de cada capa. 
                              Ej: [{'units': 1024, 'activation': 'tanh', 'dropout': 0.25}, {...}]
        """
        self.model = Sequential()
        
        # Primera capa: Flatten (Aplanar las imágenes)
        self.model.add(Flatten(input_shape=self.input_shape))
        
        # Agregar capas ocultas configuradas
        for layer in layers_config:            
            if layer['activation'] == 'leakyrelu':
                self.model.add(Dense(layer['units']))
                self.model.add(LeakyReLU(alpha=0.01))
            else:
                self.model.add(Dense(layer['units'], activation=layer['activation']))

            if batch_normalization:
                self.model.add(BatchNormalization())

            if 'dropout' in layer:
                self.model.add(Dropout(layer['dropout']))
        
        # Capa de salida (softmax para clasificación multiclase)
        self.model.add(Dense(self.output_size, activation='softmax'))

    def compile_model(self, optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']):
        """
        Compila el modelo con el optimizador, la función de pérdida y las métricas indicadas.
        """
        if self.model:
            self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
            self.model.summary()
        else:
            raise ValueError("El modelo no está construido. Llama a build_model primero.")
    
    def get_model(self):
        """
        Devuelve el modelo generado.
        """
        return self.model

    def fit_model(self, train=train, validation=validation, epochs=10, batch_size=32, callbacks=[OurCustomCallback()]):
        """
        Entrena el modelo con 10 épocas por default.
        """
        return PlotHistory(self.model.fit(
            train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=validation,
            callbacks=callbacks,
        ))

## Modelo MLP 1024 - 512 - 256

In [ ]:
# Configuración de capas del MLP
layers_config = [
    {'units': 1024, 'activation': 'relu', 'dropout': 0.25},
    {'units': 512, 'activation': 'relu', 'dropout': 0.25},
    {'units': 256, 'activation': 'relu', 'dropout': 0.25}
]

mlp_model_builder = MLPBuilder()
mlp_model_builder.build_model(layers_config)
mlp_model_builder.compile_model()

In [ ]:
# Entrena el modelo y lo plotea
mlp_model_builder.fit_model()

## Conclusiones

Este primer modelo MLP que probamos no está alcanzando una buena precisión, tanto en el conjunto de entrenamiento como en el de validación. La precisión se mantiene baja (~0.26) y la pérdida es relativamente alta. 

El número de neuronas (1024, 512, 256) podría ser demasiado grande para el problema. Vamos a reducirlas para evitar el sobreajuste, dado que el modelo parece estar teniendo dificultades para generalizar.



## Modelo MLP 256 - 128

In [ ]:
# Configuración de capas del MLP
layers_config = [
    {'units': 256, 'activation': 'relu', 'dropout': 0.25},
    {'units': 128, 'activation': 'relu', 'dropout': 0.25},
]

mlp_model_builder = MLPBuilder()
mlp_model_builder.build_model(layers_config)
mlp_model_builder.compile_model()

In [ ]:
# Entrena el modelo y lo plotea
mlp_model_builder.fit_model()

## Conclusiones

El nuevo modelo simplificado muestra una mejora en la precisión, aunque sigue habiendo espacio para optimizarlo. Actualmente, la precisión en el conjunto de validación se estabiliza entre un 30% y 35%, lo que indica que el modelo aún no está capturando de manera efectiva todas las características del conjunto de datos. Se puede observar que en la época 8 el modelo alcanza su límite de rendimiento, comenzando a presentar signos de sobreajuste.

## Modelo MLP con LeakyReLU y Batch Normalization

In [ ]:
layers_config = [
    {'units': 256, 'activation': 'leakyrelu', 'dropout': 0.15},
    {'units': 128, 'activation': 'leakyrelu', 'dropout': 0.15},
]

mlp_model_builder = MLPBuilder()
mlp_model_builder.build_model(layers_config, batch_normalization=True)
mlp_model_builder.compile_model()

In [ ]:
# Entrena el modelo y lo plotea
mlp_model_builder.fit_model()